In [1]:
import cohere  
from openai import OpenAI
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urlencode
import re
import pandas as pd


In [2]:
client = OpenAI(
    api_key="sk-dP3OLsFmsPb2kgfFaeYO5im3fx9KZkLtVBsMwRqMwvzsdQtI",  base_url="https://api.chatanywhere.tech"# This is the default and can be omitted
)

In [79]:
query = """lakers vs nuggets 2023 box score playoff game 1"""

In [26]:
encoded_query = urlencode({"q": query})
encoded_query

'q=lakers+vs+nuggets+2023+box+score+playoff+game+1'

In [27]:
#maybe free to use google query, but the content is really ill-formated.
url =f"https://www.google.com/search?{encoded_query}&gs_lcrp=EgZjaHJvbWUyBggAEEUYOTIHCAEQIRigATIHCAIQIRigATIHCAMQIRigATIHCAQQIRigATIHCAUQIRiPAjIHCAYQIRiPAtIBCDIxODlqMGo3qAIAsAIA&sourceid=chrome&ie=UTF-8&gl=us&hl=en"

In [61]:
# Function to clean up the data
def clean_data(text):
    # Remove newline characters and excess spaces
    # Replaces multiple spaces and newlines with one space
    text = re.sub(r'\s+', ' ', text)
    # Remove unwanted characters like \n, \t, or other non-printable characters
    text = re.sub(r'[\n\t\r]+', ' ', text)
    # Additional cleaning if necessary (e.g., remove '×' and other unwanted symbols)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Removes non-ASCII characters
    return text.strip()



def get_soup(url):
    try:
        # Fetch the webpage
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
        
        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract and return the text content
        content = soup.get_text(separator="\n").strip()
        content = clean_data(content)
        
        return soup, content
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None, None

In [62]:
soup, content = get_soup(url)

In [37]:
links = [a.get("href") for a in soup.find_all("a") if a.get("href")]

In [38]:
links = [link for link in links if "https://" in link]
links

['/url?q=https://maps.google.com/maps%3Fq%3Dlakers%2Bvs%2Bnuggets%2B2023%2Bbox%2Bscore%2Bplayoff%2Bgame%2B1%26gs_lcrp%3DEgZjaHJvbWUyBggAEEUYOTIHCAEQIRigATIHCAIQIRigATIHCAMQIRigATIHCAQQIRigATIHCAUQIRiPAjIHCAYQIRiPAtIBCDIxODlqMGo3qAIAsAIA%26gl%3Dus%26hl%3Den%26um%3D1%26ie%3DUTF-8%26ved%3D1t:200713%26ictx%3D111&opi=89978449&sa=U&ved=0ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQiaAMCAooBQ&usg=AOvVaw2glitLeovkQCM1RT8e1uMG',
 '/url?q=https://www.nba.com/game/lal-vs-den-0042200311/box-score&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECA4QAg&usg=AOvVaw2XOG8dpMA6nH2yQyU-B0JA',
 '/url?q=https://www.nba.com/game/den-vs-lal-0042200313/box-score&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECAcQAg&usg=AOvVaw3OpRxuOAL1cmI4-EAh2nQN',
 '/url?q=https://www.espn.com/nba/game/_/gameId/401547684/nuggets-lakers&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECA0QAg&usg=AOvVaw3nRRX6LCxGjsuPN0ClFqz9',
 '/url?q=https://www.espn.com/nba/boxscore/_/gameId/401547683&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECAwQAg&usg=A

In [42]:
def get_potential_urls(text, keyword="https://"):
    # Find the position of the keyword
    start_index = text.find(keyword)
    # Extract substring starting from the keyword
    if start_index != -1:  # Check if the keyword exists in the string
        result = text[start_index:]
        return result
    else:
        print(f"Keyword '{keyword}' not found in the text.")

In [44]:
links = [get_potential_urls(link) for link in links]
links

['https://maps.google.com/maps%3Fq%3Dlakers%2Bvs%2Bnuggets%2B2023%2Bbox%2Bscore%2Bplayoff%2Bgame%2B1%26gs_lcrp%3DEgZjaHJvbWUyBggAEEUYOTIHCAEQIRigATIHCAIQIRigATIHCAMQIRigATIHCAQQIRigATIHCAUQIRiPAjIHCAYQIRiPAtIBCDIxODlqMGo3qAIAsAIA%26gl%3Dus%26hl%3Den%26um%3D1%26ie%3DUTF-8%26ved%3D1t:200713%26ictx%3D111&opi=89978449&sa=U&ved=0ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQiaAMCAooBQ&usg=AOvVaw2glitLeovkQCM1RT8e1uMG',
 'https://www.nba.com/game/lal-vs-den-0042200311/box-score&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECA4QAg&usg=AOvVaw2XOG8dpMA6nH2yQyU-B0JA',
 'https://www.nba.com/game/den-vs-lal-0042200313/box-score&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECAcQAg&usg=AOvVaw3OpRxuOAL1cmI4-EAh2nQN',
 'https://www.espn.com/nba/game/_/gameId/401547684/nuggets-lakers&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECA0QAg&usg=AOvVaw3nRRX6LCxGjsuPN0ClFqz9',
 'https://www.espn.com/nba/boxscore/_/gameId/401547683&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECAwQAg&usg=AOvVaw2kjaFA92q2sRfagzR3JZnS',
 'htt

In [49]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": """don't use your own knowledge, rely on my data""",
        },
        {
            "role": "user",
            "content": f"""based on {links}, which elements are most related to the query:{query}?
            Return your answer as an array [url1, url2]
            """,
        }
    ],
    model="gpt-4o",
)
res = chat_completion.choices[0].message.content
print(res)

Based on the given URLs and their context, the elements most related to the query "lakers vs nuggets 2023 box score playoff game 1" are:

1. 'https://www.nba.com/game/lal-vs-den-0042200311/box-score&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECA4QAg&usg=AOvVaw2XOG8dpMA6nH2yQyU-B0JA'
2. 'https://www.espn.com/nba/game/_/gameId/401547684/nuggets-lakers&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECA0QAg&usg=AOvVaw3nRRX6LCxGjsuPN0ClFqz9'

So, the array is:

[ 'https://www.nba.com/game/lal-vs-den-0042200311/box-score&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECA4QAg&usg=AOvVaw2XOG8dpMA6nH2yQyU-B0JA', 'https://www.espn.com/nba/game/_/gameId/401547684/nuggets-lakers&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECA0QAg&usg=AOvVaw3nRRX6LCxGjsuPN0ClFqz9' ]


takeaway, LLM not always return what you want, maybe a better idea is to test one by one

In [50]:
def call_openai(client, messages, model="gpt-4o"):
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
    )
    return chat_completion.choices[0].message.content

In [57]:
potential_links = []
for link in links:
    messages = [
        {
            "role": "system",
            "content": """don't use your own knowledge, rely on my data""",
        },
        {
            "role": "user",
            "content": f"""based on {link}, do you think it is related to the query:{query}?
                Return your answer yes or no
                """,
        }
    ]
    res = call_openai(client, messages)
    print(res ,link)
    if res.lower() in ['yes', 'yes.']:
        potential_links.append(link)

Yes https://maps.google.com/maps%3Fq%3Dlakers%2Bvs%2Bnuggets%2B2023%2Bbox%2Bscore%2Bplayoff%2Bgame%2B1%26gs_lcrp%3DEgZjaHJvbWUyBggAEEUYOTIHCAEQIRigATIHCAIQIRigATIHCAMQIRigATIHCAQQIRigATIHCAUQIRiPAjIHCAYQIRiPAtIBCDIxODlqMGo3qAIAsAIA%26gl%3Dus%26hl%3Den%26um%3D1%26ie%3DUTF-8%26ved%3D1t:200713%26ictx%3D111&opi=89978449&sa=U&ved=0ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQiaAMCAooBQ&usg=AOvVaw2glitLeovkQCM1RT8e1uMG
Yes https://www.nba.com/game/lal-vs-den-0042200311/box-score&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECA4QAg&usg=AOvVaw2XOG8dpMA6nH2yQyU-B0JA
Yes https://www.nba.com/game/den-vs-lal-0042200313/box-score&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECAcQAg&usg=AOvVaw3OpRxuOAL1cmI4-EAh2nQN
Yes https://www.espn.com/nba/game/_/gameId/401547684/nuggets-lakers&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECA0QAg&usg=AOvVaw3nRRX6LCxGjsuPN0ClFqz9
Yes https://www.espn.com/nba/boxscore/_/gameId/401547683&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECAwQAg&usg=AOvVaw2kjaFA92q2sRfagzR3JZnS
Yes h

In [65]:
all_soups = []
all_contents = []
for link in potential_links:
    soup, content = get_soup(link)
    if soup:
        all_soups.append(soup)
    if content:
        all_contents.append(content)

An error occurred: 404 Client Error: Not Found for url: https://maps.google.com/maps%3Fq%3Dlakers%2Bvs%2Bnuggets%2B2023%2Bbox%2Bscore%2Bplayoff%2Bgame%2B1%26gs_lcrp%3DEgZjaHJvbWUyBggAEEUYOTIHCAEQIRigATIHCAIQIRigATIHCAMQIRigATIHCAQQIRigATIHCAUQIRiPAjIHCAYQIRiPAtIBCDIxODlqMGo3qAIAsAIA%26gl%3Dus%26hl%3Den%26um%3D1%26ie%3DUTF-8%26ved%3D1t:200713%26ictx%3D111&opi=89978449&sa=U&ved=0ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQiaAMCAooBQ&usg=AOvVaw2glitLeovkQCM1RT8e1uMG
An error occurred: 403 Client Error: Forbidden for url: https://www.espn.com/nba/game/_/gameId/401547684/nuggets-lakers&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECA0QAg&usg=AOvVaw3nRRX6LCxGjsuPN0ClFqz9
An error occurred: 403 Client Error: Forbidden for url: https://www.espn.com/nba/boxscore/_/gameId/401547683&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECAwQAg&usg=AOvVaw2kjaFA92q2sRfagzR3JZnS
An error occurred: 404 Client Error: Not Found for url: https://www.basketball-reference.com/playoffs/2023-nba-western-conference-finals-lakers

In [69]:
get_soup("https://www.espn.com/nba/boxscore/_/gameId/401547683&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECAwQAg&usg=AOvVaw2kjaFA92q2sRfagzR3JZnS")

An error occurred: 403 Client Error: Forbidden for url: https://www.espn.com/nba/boxscore/_/gameId/401547683&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECAwQAg&usg=AOvVaw2kjaFA92q2sRfagzR3JZnS


(None, None)

don't know why requests returns 403

In [75]:
# Set up Selenium WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = "https://www.espn.com/nba/boxscore/_/gameId/401547683&sa=U&ved=2ahUKEwia0dCI0u2JAxWGbPUHHbgfJ1IQFnoECAwQAg&usg=AOvVaw2kjaFA92q2sRfagzR3JZnS"
driver.get(url)

# Extract the page source and process it
content = driver.page_source

soup = BeautifulSoup(content, 'html.parser')
content = soup.get_text(separator="\n").strip()
content = clean_data(content)

driver.quit()


In [77]:
content

"Nuggets 119-108 Lakers (May 20, 2023) Box Score - ESPN Skip to main content Skip to navigation Top Events NFL NCAAF NBA NHL NCAAM NCAAW UFC Top Soccer PGA Tour LPGA Tour NBA G League Tennis (M) Tennis (W) F1 NASCAR PLL (Lacrosse) Women's NCAA Volleyball WNBA MLB NBA Nov 19 Nov 20 Nov 21 Nov 21 Gamecast Box Score NO 100 CLE 128 Final Gamecast Box Score CHI 106 MIL 122 Final Gamecast Box Score IND 113 HOU 130 Final Gamecast Box Score PHI 111 MEM 117 Final Gamecast Box Score POR 99 OKC 109 Final Gamecast Box Score NY 138 PHX 122 Final Gamecast Box Score ATL 97 GS 120 Final Gamecast Box Score ORL 93 LAC 104 Final Full Scoreboard   ESPN NFL NFL NBA NBA MLB MLB NCAAF NCAAF NHL NHL Soccer Soccer More Sports More Sports ESPN Plus ESPN BET Watch Watch Fantasy Fantasy NEW! Find where to watch all of your favorite sports! Search Profile Management West Finals - Game 3 DEN leads series 3-0 Nuggets 53-29 , 19-22 away 119 Winner Icon Denver Nuggets Final 1 2 3 4 T DEN 32 26 26 35 119 LAL 20 35 27 2

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "don't use your own knowledge. use the data I gave you",
        },
        {
            "role": "user",
            "content": f"""answer the question: {query}.
                            from the data {content}, """
        }
    ],
    model="gpt-4o",
)
response1 = chat_completion.choices[0].message.content
response1

'In Game 1 of the 2023 playoff series between the Denver Nuggets and the Los Angeles Lakers, the Nuggets won with a score of 119 to 108. The score by quarters for the Nuggets was 32, 26, 26, and 35, and for the Lakers, it was 20, 35, 27, and 26. Key players for the Denver Nuggets included Nikola Jokic with a significant contribution of 24 points, Jamal Murray with 37 points, and Kentavious Caldwell-Pope scoring 17 points. For the Los Angeles Lakers, key performances came from Anthony Davis with 28 points, LeBron James with 23 points, and Austin Reaves also contributing 23 points. The game was held at crypto.com Arena in Los Angeles, CA.'

In [82]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "don't use your own knowledge. use the data I gave you",
        },
        {
            "role": "user",
            "content": f"""can you get the box score {query}.
                            from the data {content}, """
        }
    ],
    model="gpt-4o",
)
response1 = chat_completion.choices[0].message.content
response1

"Here is the box score for the Denver Nuggets vs. Los Angeles Lakers Game 3 of the playoffs on May 20, 2023:\n\n**Team Score:**\n- Denver Nuggets: 119\n- Los Angeles Lakers: 108\n\n**Quarterly Breakdown:**\n- Denver Nuggets: 32 (1st), 26 (2nd), 26 (3rd), 35 (4th)\n- Los Angeles Lakers: 20 (1st), 35 (2nd), 27 (3rd), 26 (4th)\n\n**Denver Nuggets Stats:**\n- **Starters:**\n  - Aaron Gordon: 32 MIN, 2-5 FG, 0-1 3PT, 3-4 FT, 3 REB, 4 AST, 7 PTS\n  - Michael Porter Jr.: 38 MIN, 4-11 FG, 4-10 3PT, 2-2 FT, 10 REB, 6 AST, 14 PTS\n  - Nikola Jokic: 38 MIN, 9-19 FG, 2-5 3PT, 4-7 FT, 6 REB, 8 AST, 24 PTS\n  - Jamal Murray: 42 MIN, 15-29 FG, 5-11 3PT, 2-2 FT, 7 REB, 6 AST, 37 PTS\n  - Kentavious Caldwell-Pope: 34 MIN, 6-10 FG, 4-7 3PT, 1-2 FT, 3 REB, 17 PTS\n- **Bench:**\n  - Jeff Green: 23 MIN, 2-3 FG, 1-2 3PT, 4 REB, 5 PTS\n  - Bruce Brown: 29 MIN, 6-11 FG, 1-5 3PT, 2-2 FT, 5 REB, 5 AST, 15 PTS\n  - Christian Braun: 3 MIN, 1 REB, 0 PTS\n  - Others: DNP-COACH'S DECISION\n\n**Los Angeles Lakers Sta